# CSC411 Assignment 3

## Helper Functions for Part 2

In [30]:
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.image as mpimg
import os
import urllib
# Constants
# -----------------------------------------------------------------------------

In [31]:
# Functions used in multiple parts
# -----------------------------------------------------------------------------
def getNews(seed = 0):
    
    fakeFile = open("clean_fake.txt",'r')
    fakeNews = fakeFile.read().splitlines()
    fakeFile.close()
    realFile = open("clean_real.txt",'r')
    realNews = realFile.read().splitlines()
    realFile.close()
    
    np.random.shuffle(fakeNews)
    np.random.shuffle(realNews)
    
    return fakeNews, realNews

In [95]:
# test
fakeNews, realNews = getNews()


['who to blame for president trump',
 'why this orthodox rabbi supports trump',
 'the real reason why the clinton email scandal is back has nothing to do with helping trump',
 'trump protest feed clinton supporters take to the streets because their team didnt win',
 'saudi arabia warns trump on blocking oil imports ft',
 'la sortie du dernier livre in dit du gorafi clipse totalement la victoire de donald trump le gorafi',
 'it literally hurts my brain to read the economic idiocy emitted by trumpkins libertarian',
 'former u treasury secretary just warned trump could be assassinated',
 'trump spokesperson katrina pierson caught in blatant lie threatened with lawsuit',
 'dear liberals trump is right',
 'project veritas video 4 20k bribery to dnc chaos at trump rallies',
 'october surprise abc uncovers millions of payments from russia to trump',
 'america post trump widening divide',
 'trump jr suspiciously helps arizona woman push stalled car uses it as a photo op',
 'jewish home touts t

In [33]:
def dataSetSplit(fakeNews, realNews,seed = 0):
    
    trainingSet ={'real': realNews[:int(0.7*len(realNews))], 'fake':fakeNews[:int(0.7*len(fakeNews))]} 
    validationSet = {'real':realNews[int(0.7*len(realNews)):int(0.85*len(realNews))], 'fake':fakeNews[int(0.7*len(fakeNews)):int(0.85*len(fakeNews))]}
    testSet = {'real': realNews[int(0.85*len(realNews)):] , 'fake': fakeNews[int(0.85*len(fakeNews)):]}

    return trainingSet, validationSet, testSet

In [34]:
# test
trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)

In [35]:
# test
trainingSet

{'fake': ['mother and daughter remind msnbc reporter that trump has black supporters',
  'trump campaign for celebrity',
  'how trump happened',
  'stop trump mixtape a protest song collection',
  'greens question trump victory',
  'shocking nostradamus predicted donald trump rise to power',
  'the color revolution against trump',
  'trump to deport everyone who is not native american',
  'breaking police catch iowa man who executed two officers and he a trump deplorable if you only news',
  'mainstream media ignoring 75 lawsuits against trump to focus on clinton emails',
  'dilbert creator scott adams endorses trump says hillary is a bully',
  'trump calls black man at rally a thug kicks him out except there one big problem',
  'new play vicu a portrays trump in emperor new clothes tale',
  'comment on dilbert cartoonist hypnotist scott adams in 2015 predicted a hillary health crisis by free the trumprolled',
  'trump family already sworn to secrecy about faked moon landings more soon

In [36]:
def count(trainingSet, validationSet, testSet):
    trainingCount = {'real':{}, 'fake':{}}
    validationCount = {'real':{}, 'fake':{}}
    testCount = {'real':{}, 'fake':{}}
    
    for rof in ['real','fake']:
        for news in trainingSet[rof]:
            for word in set(news.split(' ')):
                if not word in trainingCount[rof].keys():
                    trainingCount[rof][word] = 1
                else:
                    trainingCount[rof][word] += 1
        
        for news in validationSet[rof]:
            for word in set(news.split(' ')):
                if not word in validationCount[rof].keys():
                    validationCount[rof][word] = 1
                else:
                    validationCount[rof][word] += 1
        
        for news in testSet[rof]:
            for word in set(news.split(' ')):
                if not word in testCount[rof].keys():
                    testCount[rof][word] = 1
                else:
                    testCount[rof][word] += 1

    trainingRealCount = len(trainingSet['real'])
    trainingFakeCount = len(trainingSet['fake'])
    validationRealCount = len(validationSet['real'])
    validationFakeCount = len(validationSet['fake'])
    testRealCount = len(testSet['real'])
    testFakeCount = len(testSet['fake'])
    
    return trainingRealCount,trainingFakeCount,validationRealCount,validationFakeCount,testRealCount,testFakeCount, trainingCount, validationCount, testCount


In [37]:
# test
trainingRealCount,trainingFakeCount,validationRealCount,validationFakeCount,testRealCount,testFakeCount, trainingCount, validationCount, testCount = count(trainingSet, validationSet, testSet)

In [38]:
# test
trainingCount

{'fake': {'pardon': 1,
  'pide': 1,
  'colleges': 1,
  'four': 1,
  'debunks': 1,
  'protest': 5,
  'sleep': 1,
  'hating': 1,
  'battleground': 2,
  'hate': 5,
  'assembled': 1,
  'whose': 1,
  'votes': 12,
  'voter': 22,
  'founder': 1,
  'vandal': 1,
  'tweet': 5,
  'crush': 1,
  'edward': 1,
  'voted': 2,
  'under': 1,
  'teaching': 1,
  'sway': 1,
  'regrets': 1,
  'divergent': 1,
  'evangelical': 1,
  'risk': 1,
  'rise': 3,
  'smack': 1,
  'mania': 1,
  'every': 4,
  'politician': 1,
  'bringing': 1,
  'school': 1,
  'hath': 1,
  'solution': 1,
  'solid': 1,
  'pardons': 1,
  'math': 1,
  'unhinged': 1,
  'deplorable': 4,
  'clothes': 1,
  'uncovers': 1,
  'force': 1,
  'tired': 3,
  'phones': 1,
  'warns': 6,
  'pilger': 2,
  '270': 3,
  'elegant': 1,
  'crave': 8,
  'persuade': 1,
  '49': 1,
  'n': 4,
  'draintheswamp': 1,
  'machines': 6,
  'theory': 1,
  'even': 7,
  'neo': 1,
  'brexit': 3,
  'nwo': 1,
  'hide': 1,
  'reopens': 2,
  'beaten': 5,
  'fingers': 1,
  'liberty':

In [64]:
def probability(trainingRealCount,trainingFakeCount, trainingCount, m, pHat):
    trainingProbability = {'real': {}, 'fake': {}}
    for word in trainingCount['real'].keys():
        trainingProbability['real'][word] = (trainingCount['real'][word]+m*pHat)/float(trainingRealCount+m)
    for word in trainingCount['fake'].keys():
        trainingProbability['fake'][word] = (trainingCount['fake'][word]+m*pHat)/float(trainingFakeCount+m)
    
    realProbability = float(trainingRealCount)/(trainingRealCount+trainingFakeCount)
    fakeProbability = float(trainingFakeCount)/(trainingRealCount+trainingFakeCount)
    
    return trainingProbability, realProbability, fakeProbability

In [66]:
# test
mReal = 0.1
pHatReal = 0.0001
mFake = 0.1
pHatFake = 0.0001
trainingProbability, realProbability, fakeProbability = probability(trainingRealCount,trainingFakeCount, trainingCount, m, pHat)

In [67]:
# test
trainingProbability

{'fake': {'pardon': 0.002202411628675256,
  'hats': 0.0011012113203391698,
  'pide': 0.0011012113203391698,
  'all': 0.017619215945380465,
  'four': 0.0011012113203391698,
  'debunks': 0.0011012113203391698,
  'protest': 0.005506012553683514,
  'sleep': 0.0011012113203391698,
  'deri': 0.0011012113203391698,
  'hating': 0.0011012113203391698,
  'battleground': 0.002202411628675256,
  'hate': 0.007708413170355687,
  'assembled': 0.0011012113203391698,
  'votes': 0.00991081378702786,
  'kommonsentsjane': 0.0011012113203391698,
  'dazu': 0.0011012113203391698,
  'tweet': 0.0033036119370113423,
  'edward': 0.002202411628675256,
  'voted': 0.0011012113203391698,
  'under': 0.0033036119370113423,
  'teaching': 0.0011012113203391698,
  'sway': 0.0011012113203391698,
  'regrets': 0.0011012113203391698,
  'divergent': 0.0011012113203391698,
  'convening': 0.0011012113203391698,
  'risk': 0.0011012113203391698,
  'rise': 0.0011012113203391698,
  'mania': 0.0011012113203391698,
  'every': 0.00550

In [73]:
def getAccuracy(trainingProbability,realProbability, fakeProbability, targetSet, m, pHat):
    accuracy = 0
    for rof in ['real', 'fake']:
        for numNews in range(len(targetSet[rof])):
            logProb = 0   
            for word in trainingProbability['real'].keys():
                if word in targetSet[rof][numNews].split(' '):
                    logProb += log(trainingProbability['real'][word])
                else:
                    logProb += log(1-trainingProbability['real'][word])
            for word in targetSet[rof][numNews].split(' '):
                if not word in trainingProbability['real'].keys():
                    logProb += log(pHat)
            logProb += log(realProbability)
            realProb = logProb

            logProb = 0 
            for word in trainingProbability['fake'].keys():
                if word in targetSet[rof][numNews].split(' '):
                    logProb += log(trainingProbability['fake'][word])
                else:
                    logProb += log(1-trainingProbability['fake'][word])
            for word in targetSet[rof][numNews].split(' '):
                if not word in trainingProbability['fake'].keys():
                    logProb += log(pHat)
            logProb += log(fakeProbability)
            fakeProb = logProb
            
            if rof == 'real':
                if realProb > fakeProb:
                    accuracy += 1
            else:
                if realProb < fakeProb:
                    accuracy += 1
                    
    return accuracy / (float(len(targetSet['fake']))+float(len(targetSet['real'])))


In [74]:
# test
np.random.seed(0)

fakeNews, realNews = getNews()
trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
trainingRealCount,trainingFakeCount,validationRealCount, \
validationFakeCount,testRealCount,testFakeCount, trainingCount, \
validationCount, testCount = count(trainingSet, validationSet, testSet)

m = 0.1
pHat = 0.0001
trainingProbability, realProbability, fakeProbability = \
probability(trainingRealCount,trainingFakeCount, trainingCount, m,pHat)

getAccuracy(trainingProbability,realProbability, fakeProbability, validationSet, m, pHat)

0.8795918367346939

## Part 2

In [89]:
def part2(seed = 0):
    np.random.seed(seed)

    fakeNews, realNews = getNews()
    trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
    trainingRealCount,trainingFakeCount,validationRealCount, \
    validationFakeCount,testRealCount,testFakeCount, trainingCount, \
    validationCount, testCount = count(trainingSet, validationSet, testSet)
    
    m = 0.1
    pHat = 0.0001
    trainingProbability, realProbability, fakeProbability = \
    probability(trainingRealCount,trainingFakeCount, trainingCount, m,pHat)

    print getAccuracy(trainingProbability,realProbability, fakeProbability, validationSet, m, pHat)
    print getAccuracy(trainingProbability,realProbability, fakeProbability, testSet, m, pHat)
    print getAccuracy(trainingProbability,realProbability, fakeProbability, trainingSet, m, pHat)

## Helper Functions for Part 4

In [101]:
def generateWordBase(trainingSet):
    wordBase = []
    for rof in trainingSet.keys():
        for news in trainingSet[rof]:
            for word in news.split(' '):
                wordBase.append(word)
    return set(wordBase)

In [102]:
# test
np.random.seed(0)
fakeNews, realNews = getNews()
trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)
print generateWordBase(trainingSet)

set(['pardon', 'hats', 'pide', 'bosses', 'four', 'obstruction', 'protest', 'universe', 'controversial', 'aides', 'battleground', 'hate', 'assembled', 'marching', 'stinks', 'looking', 'votes', 'voter', 'dazu', 'paris', 'tweet', 'tonya', 'cashin', 'amtsantritt', 'activist', 'voted', 'kidman', 'under', 'teaching', 'condemns', 'whack', 'divergent', 'softener', 'evangelical', 'risk', 'dismal', '6', 'rise', 'populist', 'shoot', 'advisors', 'mania', 'every', 'jack', 'govern', 'affect', 'exporters', 'disneys', 'trumpisms', 'tickets', 'school', 'publicly', 'metals', 'called', 'skills', 'supports', 'companies', 'solution', 'piling', 'pardons', 'red', 'unhinged', 'deplorable', 'heading', 'triumph', 'clothes', 'jeff', 'achievements', 'force', 'leaders', 'tired', 'miller', 'warns', 'guidelines', 'bridgitte', 'holocaust', 'chef', 'talks', 'second', 'street', '49', 'streep', 'n', 'loathing', 'seth', 'machines', 'monster', 'blue', 'army', 'brexit', 'nwo', 'hide', 'reopens', 'religious', 'horcruxes', '

In [45]:
def generateTrainingData(trainingSet, validationSet, testSet):
    
    
    training = {'real':{}, 'fake':{}}
    validationCount = {'real':{}, 'fake':{}}
    testCount = {'real':{}, 'fake':{}}
    
    for rof in ['real','fake']:
        for news in trainingSet[rof]:
            for word in set(news.split(' ')):
                if not word in trainingCount[rof].keys():
                    trainingCount[rof][word] = 1
                else:
                    trainingCount[rof][word] += 1
        
        for news in validationSet[rof]:
            for word in set(news.split(' ')):
                if not word in validationCount[rof].keys():
                    validationCount[rof][word] = 1
                else:
                    validationCount[rof][word] += 1
        
        for news in testSet[rof]:
            for word in set(news.split(' ')):
                if not word in testCount[rof].keys():
                    testCount[rof][word] = 1
                else:
                    testCount[rof][word] += 1

    trainingRealCount = len(trainingSet['real'])
    trainingFakeCount = len(trainingSet['fake'])
    validationRealCount = len(validationSet['real'])
    validationFakeCount = len(validationSet['fake'])
    testRealCount = len(testSet['real'])
    testFakeCount = len(testSet['fake'])

Accuracy on real news:
0.928813559322


## Part 4


In [46]:
def part4(seed = 0):
    return

Accuracy on fake news:
0.805128205128


In [92]:
np.random.seed(0)
fakeNews, realNews = getNews()
trainingSet, validationSet, testSet = dataSetSplit(fakeNews, realNews)


{'fake': ['nc trump supporter hangs clinton in effigy at rally says she deserves prison',
  'trump teaches his supporters how to vote correctly',
  'what does a trump victory mean for africa',
  'democrats plan to burn down u if trump elected',
  'texas official sees record breaking turnout and it all for trump',
  'bernie sanders surprise speech outside the white house on rejecting dakota pipeline trump',
  'israeli supporters of donald trump rally at jerusalem forever event in israel capital city',
  'the man behind the trump rally disturbance in reno',
  'when trump attacked the khans he didnt know what a gold star family was',
  'speculation trump promotes nsa boss rogers to dni because he leaked the clinton emails',
  'trump froh dass er sich ab morgen endlich nicht mehr zur ckhalten muss',
  'the fbi clinton email investigation has shifted poll numbers significantly in trump favor',
  'russia warns us is now a colour revolution active zone due to trump victory',
  'dr duke and pa